## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
#from config import g_key
import os
os.chdir('..')

from config import g_key
#print(g_key)
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Dingle,10.9995,122.6711,80.24,83,89,13.67,PH,overcast clouds
1,1,Dasoguz,41.8363,59.9666,32.00,86,100,6.91,TM,overcast clouds
2,2,Severo-Kurilsk,50.6789,156.1250,12.60,86,71,11.16,RU,broken clouds
3,3,Lanigan,51.8501,-105.0344,-0.40,78,100,8.05,CA,overcast clouds
4,4,Punta Arenas,-53.1500,-70.9167,50.00,62,20,19.57,CL,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
1,1,Dasoguz,41.8363,59.9666,32.00,86,100,6.91,TM,overcast clouds
2,2,Severo-Kurilsk,50.6789,156.1250,12.60,86,71,11.16,RU,broken clouds
4,4,Punta Arenas,-53.1500,-70.9167,50.00,62,20,19.57,CL,few clouds
7,7,Bluff,-46.6000,168.3333,58.41,67,100,9.19,NZ,overcast clouds
8,8,Mantua,45.1667,10.7833,46.00,95,93,1.99,IT,overcast clouds
9,9,Camacha,33.0833,-16.3333,55.40,71,75,14.97,PT,broken clouds
10,10,Oranjemund,-28.5500,16.4333,62.20,93,61,11.48,NaN,broken clouds
11,11,Tuatapere,-46.1333,167.6833,67.17,52,100,5.95,NZ,overcast clouds
12,12,Bela,25.9333,81.9833,54.77,72,69,5.19,IN,broken clouds
13,13,Katsuura,35.1333,140.3000,46.40,34,20,8.05,JP,few clouds


In [5]:
print("Verification for the max " + str(preferred_cities_df["Max Temp"].max()))
print("Verification for the min " + str(preferred_cities_df["Max Temp"].min()))

Verification for the max 69.89
Verification for the min 10.15


In [6]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.isna().sum())
print("---")
print(preferred_cities_df.count())
print("---")
print(preferred_cities_df.head(2))

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                3
Current Description    0
dtype: int64
---
City_ID                372
City                   372
Lat                    372
Lng                    372
Max Temp               372
Humidity               372
Cloudiness             372
Wind Speed             372
Country                369
Current Description    372
dtype: int64
---
   City_ID            City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
1        1         Dasoguz  41.8363   59.9666      32.0        86         100   
2        2  Severo-Kurilsk  50.6789  156.1250      12.6        86          71   

   Wind Speed Country Current Description  
1        6.91      TM     overcast clouds  
2       11.16      RU       broken clouds  


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].dropna().copy()
clean_df["Hotel Name"] = ""
clean_df.head(10)
#clean_df.count()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Dasoguz,TM,32.00,41.8363,59.9666,
2,Severo-Kurilsk,RU,12.60,50.6789,156.1250,
4,Punta Arenas,CL,50.00,-53.1500,-70.9167,
7,Bluff,NZ,58.41,-46.6000,168.3333,
8,Mantua,IT,46.00,45.1667,10.7833,
9,Camacha,PT,55.40,33.0833,-16.3333,
11,Tuatapere,NZ,67.17,-46.1333,167.6833,
12,Bela,IN,54.77,25.9333,81.9833,
13,Katsuura,JP,46.40,35.1333,140.3000,
14,Guerrero Negro,MX,61.93,27.9769,-114.0611,


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dasoguz,TM,32.00,overcast clouds,41.8363,59.9666,
2,Severo-Kurilsk,RU,12.60,broken clouds,50.6789,156.1250,
4,Punta Arenas,CL,50.00,few clouds,-53.1500,-70.9167,
7,Bluff,NZ,58.41,overcast clouds,-46.6000,168.3333,
8,Mantua,IT,46.00,overcast clouds,45.1667,10.7833,
9,Camacha,PT,55.40,broken clouds,33.0833,-16.3333,
10,Oranjemund,NaN,62.20,broken clouds,-28.5500,16.4333,
11,Tuatapere,NZ,67.17,overcast clouds,-46.1333,167.6833,
12,Bela,IN,54.77,broken clouds,25.9333,81.9833,
13,Katsuura,JP,46.40,few clouds,35.1333,140.3000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = pd.DataFrame(hotel_df)
#clean_hotel_df.isna().sum()
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dasoguz,TM,32.00,overcast clouds,41.8363,59.9666,Hotel Erik
2,Severo-Kurilsk,RU,12.60,broken clouds,50.6789,156.1250,Kak Doma
4,Punta Arenas,CL,50.00,few clouds,-53.1500,-70.9167,Hotel Dreams Del Estrecho
7,Bluff,NZ,58.41,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
8,Mantua,IT,46.00,overcast clouds,45.1667,10.7833,Hotel la Favorita
9,Camacha,PT,55.40,broken clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
11,Tuatapere,NZ,67.17,overcast clouds,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
12,Bela,IN,54.77,broken clouds,25.9333,81.9833,Raj Bhawan
13,Katsuura,JP,46.40,few clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
14,Guerrero Negro,MX,61.93,overcast clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Description</b></dt><dd>{Current Description}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))